In [ ]:
import sys
sys.path.append('../input/timmdataset/pytorch-image-models-master')
import timm

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import matplotlib.pyplot as plt
import torchvision
from tqdm import tqdm
import cv2
from skimage import io
import time

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90, RandomCrop,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GroupKFold, StratifiedKFold
import tqdm.notebook as tq
from sklearn.model_selection import train_test_split
from scipy.special import softmax

In [ ]:
CFG = {
    'img_size': 512,
    'vit_img': 384,
    'tta': 3,
    'valid_bs': 48,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'effnet_model_b3' : ['tf_efficientnet_b3_ns_model_2.pth','tf_efficientnet_b3_ns_model_5.pth','tf_efficientnet_b3_ns_model_4.pth','tf_efficientnet_b3_ns_model_1.pth','tf_efficientnet_b3_ns_model_3.pth'],
    'effnet_model_b4' : ['tf_efficientnet_b4_ns_model_3.pth','tf_efficientnet_b4_ns_model_5.pth','tf_efficientnet_b4_ns_model_2.pth','tf_efficientnet_b4_ns_model_1.pth','tf_efficientnet_b4_ns_model_4.pth'],
    'resnet_models' : ['resnet50d_model_1.pth','resnet50d_model_4.pth','resnet50d_model_3.pth','resnet50d_model_2.pth','resnet50d_model_5.pth'],
    'resnext_models' : ['resnext50d_32x4d_model_5.pth','resnext50d_32x4d_model_3.pth','resnext50d_32x4d_model_1.pth','resnext50d_32x4d_model_2.pth','resnext50d_32x4d_model_4.pth'],
    'resnext101_models' : ['ig_resnext101_32x8d_model_1.pth','ig_resnext101_32x8d_model_2.pth','ig_resnext101_32x8d_model_3.pth','ig_resnext101_32x8d_model_4.pth','ig_resnext101_32x8d_model_5.pth'],
    'vit_models' : ['vit_base_patch16_384_model_2.pth','vit_base_patch16_384_model_5.pth','vit_base_patch16_384_model_1.pth','vit_base_patch16_384_model_3.pth','vit_base_patch16_384_model_4.pth']
}

In [ ]:
class DiseaseDatasetInference(torch.utils.data.Dataset):

    def __init__ (self, df, transform=None, opt_label=True):
        self.df = df.reset_index(drop=True).copy()
        self.transform = transform
        self.opt_label = opt_label

        if self.opt_label:
            self.data = [(row['image_id'], row['label']) for _, row in self.df.iterrows()]

        else:
            self.data = [(row['image_id']) for _, row in self.df.iterrows()]

        self.data = np.asarray(self.data)
  
    def __len__(self):
        return len(self.data)

    def __getitem__ (self, index):
            # np.random.shuffle(self.data)
        if self.opt_label:
            image_path, label = self.data[index]    
        else:
            image_path = self.data[index]

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']

        if self.opt_label == True:
            return (image, int(label))

        else:
            return image

In [ ]:
def get_inference_Vit_transforms():
    return Compose([
#             RandomCrop(CFG['vit_img'], CFG['vit_img'], p=0.5),
            CenterCrop(CFG['vit_img'], CFG['vit_img'], p=0.5),
            Resize(CFG['vit_img'], CFG['vit_img']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
            
        ], p=1.)

In [ ]:
def get_inference_transforms():
    return Compose([
#             RandomCrop(CFG['img_size'], CFG['img_size'], p=0.5),
            CenterCrop(CFG['img_size'], CFG['img_size'], p=0.5),
            Resize(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
            
        ], p=1.)

In [ ]:
df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')
PATH = '/kaggle/input/cassava-leaf-disease-classification/test_images/'

In [ ]:
test_csv = df.copy()
test_csv['image_id'] = PATH + test_csv['image_id']

test_ds = DiseaseDatasetInference(test_csv, transform=get_inference_transforms(), opt_label=False)
test_ds_vit = DiseaseDatasetInference(test_csv, transform=get_inference_Vit_transforms(), opt_label=False)

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=CFG['valid_bs'], shuffle=False, pin_memory=False) 
test_loader_vit = torch.utils.data.DataLoader(test_ds_vit, batch_size=CFG['valid_bs'], shuffle=False, pin_memory=False) 

In [ ]:
def inference (model, data_loader, device):
    preds = []
    model.eval()
    test_tqdm = tq.tqdm(data_loader, total=len(data_loader), desc="Testing", position=0, leave=True)
    for images in test_tqdm:
        images = images.to(device)
        preds.extend(model(images).detach().cpu().numpy())
    return preds

# **EfficientNet**

In [ ]:
class Effnet(nn.Module):
    def __init__(self, model_name = 'tf_efficientnet_b3_ns', pretrained = False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained = pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, 5)
        
    def forward(self, x):
        x = self.model(x)
        return x

# EfficientNet B3

In [ ]:
path_effnet = '../input/torch-my-cassava-effnetb3/'

model_name = 'tf_efficientnet_b3_ns'
effnet_preds = []
effnet_model = Effnet(model_name = model_name)
for effnet_model_name in CFG['effnet_model_b3']:
    print("Model: ", effnet_model_name)
    effnet_model.to(CFG['device'])
    effnet_model.load_state_dict(torch.load(path_effnet+effnet_model_name, map_location=torch.device(CFG['device'])))
    with torch.no_grad():
        for i in range(CFG['tta']):
            effnet_preds += [inference(effnet_model, test_loader, CFG['device'])]
effnet_preds = np.mean(effnet_preds, axis=0)

In [ ]:
effnet_outcomes_b3 = pd.concat([df['image_id'], pd.DataFrame(effnet_preds)], axis=1).sort_values(['image_id'])

# EfficientNet B4

In [ ]:
# path_effnet = '../input/torch-my-cassava-effnetb4/'


# model_name = 'tf_efficientnet_b4_ns'
# effnet_preds = []
# effnet_model = Effnet(model_name = model_name)
# for effnet_model_name in CFG['effnet_model_b4']:
#     print("Model: ", effnet_model_name)
#     effnet_model.to(CFG['device'])
#     effnet_model.load_state_dict(torch.load(path_effnet+effnet_model_name, map_location=torch.device(CFG['device'])))
#     with torch.no_grad():
#         for i in range(CFG['tta']):
#             effnet_preds += [inference(effnet_model, test_loader, CFG['device'])]
# effnet_preds = np.mean(effnet_preds, axis=0)

In [ ]:
# effnet_outcomes_b4 = pd.concat([df['image_id'], pd.DataFrame(effnet_preds)], axis=1).sort_values(['image_id'])

# **ResNet**

In [ ]:
class CustomResNet(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, 5)

    def forward(self, x):
        x = self.model(x)
        return x

# ResNet50d

In [ ]:
# resnet_preds = []
# path_resnet = '../input/torch-my-cassava-resnet50d/'

# model_name = 'resnet50d'

# resnet_model = CustomResNet(model_name = model_name)
# resnet_model.to(CFG['device'])
# for resnet_model_name in CFG['resnet_models']:
#     print("Model: ", resnet_model_name)
#     resnet_model.load_state_dict(torch.load(path_resnet + resnet_model_name, 
#                                              map_location=torch.device(CFG['device'])))
#     with torch.no_grad():
#         for i in range(CFG['tta']):
#             resnet_preds += [inference(resnet_model, test_loader, CFG['device'])]
# resnet_preds = np.mean(resnet_preds, axis=0)

In [ ]:
# resnet_outcomes = pd.concat([df['image_id'], pd.DataFrame(resnet_preds)], axis=1).sort_values(['image_id'])

# ResNext50d

In [ ]:
resnet_preds = []
path_resnet = '../input/torch-my-cassava-resnext50d/'

model_name = 'resnext50d_32x4d'

resnet_model = CustomResNet(model_name = model_name)
resnet_model.to(CFG['device'])
for resnet_model_name in CFG['resnext_models']:
    print("Model: ", resnet_model_name)
    resnet_model.load_state_dict(torch.load(path_resnet + resnet_model_name, 
                                             map_location=torch.device(CFG['device'])))
    with torch.no_grad():
        for i in range(CFG['tta']):
            resnet_preds += [inference(resnet_model, test_loader, CFG['device'])]
resnet_preds = np.mean(resnet_preds, axis=0)

In [ ]:
resnext_outcomes = pd.concat([df['image_id'], pd.DataFrame(resnet_preds)], axis=1).sort_values(['image_id'])

# ResNext 101

In [ ]:
# resnet_preds = []
# path_resnet = '../input/torch-my-cassava-resnext101/'

# model_name = 'ig_resnext101_32x8d'

# resnet_model = CustomResNet(model_name = model_name)
# resnet_model.to(CFG['device'])
# for resnet_model_name in CFG['resnext101_models']:
#     print("Model: ", resnet_model_name)
#     resnet_model.load_state_dict(torch.load(path_resnet + resnet_model_name, 
#                                              map_location=torch.device(CFG['device'])))
#     with torch.no_grad():
#         for i in range(CFG['tta']):
#             resnet_preds += [inference(resnet_model, test_loader, CFG['device'])]
# resnet_preds = np.mean(resnet_preds, axis=0)

In [ ]:
# resnext_outcomes = pd.concat([df['image_id'], pd.DataFrame(resnet_preds)], axis=1).sort_values(['image_id'])

# ViT

In [ ]:
class ViTClassifier(nn.Module):
    def __init__(self, model_name = 'vit_base_patch16_384', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, 5)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
path_vit = '../input/torch-my-cassava-vit-16-384/'


model_name = 'vit_base_patch16_384'
vit_preds = []
vit_model = ViTClassifier(model_name = model_name)
for vit_model_name in CFG['vit_models']:
    print("Model: ", vit_model_name)
    vit_model.to(CFG['device'])
    vit_model.load_state_dict(torch.load(path_vit+vit_model_name, map_location=torch.device(CFG['device'])))
    with torch.no_grad():
        for i in range(CFG['tta']):
            vit_preds += [inference(vit_model, test_loader_vit, CFG['device'])]
vit_preds = np.mean(vit_preds, axis=0)

In [ ]:
vit_outcomes = pd.concat([df['image_id'], pd.DataFrame(vit_preds)], axis=1).sort_values(['image_id'])

# Ensemble

In [ ]:
# final_preds = (effnet_outcomes_b3.drop('image_id', axis=1) * 0.25 + effnet_outcomes_b4.drop('image_id', axis=1) * 0.25 + resnet_outcomes.drop('image_id', axis=1) * 0.25 + resnext_outcomes.drop('image_id', axis=1) * 0.25).to_numpy()
final_preds = (resnext_outcomes.drop('image_id', axis=1) * 0.4 + vit_outcomes.drop('image_id', axis=1) * 0.3 + effnet_outcomes_b3.drop('image_id', axis=1) * 0.3).to_numpy()
# final_preds = (resnext_outcomes.drop('image_id', axis=1) * 0.5 + effnet_outcomes_b3.drop('image_id', axis=1) * 0.5).to_numpy()
final_preds = softmax(final_preds).argmax(1)

In [ ]:
submit = pd.DataFrame({'image_id': df['image_id'].values, 'label': final_preds})
submit.to_csv('submission.csv', index=False)